# Tweets from bioRxiv and medRxiv

Find and display events from the CSHL DOI prefix 10.1101 from the last 7 days, and find the most tweeted preprints.

In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '..')

import pandas as pd # data analysis library
import json
import mrced2 # module to run event data queries
import os # some file manipulations
import math # some number manipulations
import altair.vegalite.v3 as alt # some data visualizations
from IPython.display import Markdown as md # some markdown manipulations
from datetime import datetime, date, timedelta # some date manipulations

In [2]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
end_date = date.today()

In [3]:
ed = mrced2.eventData(email = email)
ed.buildQuery({'obj-id.prefix' : prefix, 'source': 'twitter', 'rows': 0,'from-occurred-date' : start_date, 'until-occurred-date' : end_date})

https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&obj-id.prefix=10.1101&source=twitter&rows=0&from-occurred-date=2021-05-17&until-occurred-date=2021-05-24


In [4]:
ed.runQuery(retry = 5)

Event Data query started...
API query complete  200
output file written to 1101/tweets.json


In [5]:
pages = math.ceil(ed.events.getHits() / 1000)

13241 events found


In [6]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

# find the all result pages for the search
ed = mrced2.eventData(email = email)
ed.getAllPages(pages, {'rows': 1000, 'obj-id.prefix' : prefix, 'source': 'twitter', 'from-occurred-date' : start_date, 'until-occurred-date' : end_date}, fileprefix = '1101/tweets_') 

https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-05-17&until-occurred-date=2021-05-24
Event Data query started...
API query complete  200
output file written to 1101/tweets_0000.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=6bb8f0f2-433c-478f-b20a-326c183abd30&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-05-17&until-occurred-date=2021-05-24
Event Data query started...
API query complete  200
output file written to 1101/tweets_0001.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=4aca4239-e832-4a49-bfa2-2e49789aad75&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-05-17&until-occurred-date=2021-05-24
Event Data query started...
API query complete  200
output file written to 1101/tweets_0002.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=40d36e98-8c5b-45f5-819d-5b8ff7b13756&rows=

Initialisation to look at the properties of the results.

In [7]:
jd1 = mrced2.eventRecord() # instance of a class to interpret the events
files = os.listdir('1101') # get all the filenames

jd1.mergeJsons(files, folder = '1101') # load the json event data from multiple files

failed to load preprint_tweets_2021-05-10.csv
failed to load preprint_tweets_2021-05-17.csv
failed to load .DS_Store
failed to load .gitkeep
failed to load tweets_sorted.json
failed to load preprint_tweets_2021-05-18.csv
failed to load preprint_tweets_2021-05-09.csv
failed to load preprint_tweets_2021-05-08.csv
output file written to 1101/tweets.json


In [8]:
js = json.load(open("1101/tweets.json"))
df = pd.json_normalize(js, record_path = ['message', 'events'])
gdf = df.groupby(['obj_id']).size().reset_index(name='count').sort_values('count', ascending=False)
cdf = gdf[gdf['count'] >= 5]

In [9]:
email = "info@front-matter.io"
rest = mrced2.restApi(email = email)

data = []

for index, row in cdf.iterrows():
    rest.runQuery(row)
    if rest.work is not None and date.fromisoformat(rest.work["posted"]) >= start_date:
        data.append(rest.work)
    
tdf = pd.DataFrame(data, columns=['doi','tweets','archive','subject-area','covid','title','authors','abstract','posted'])
tdf.to_csv('1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv')

tdf.head(50)

REST API query started for 10.1101/2021.05.18.21257267...
REST API query complete  200
REST API query started for 10.1101/2020.11.15.383323...
REST API query complete  200
REST API query started for 10.1101/2021.05.07.21256823...
REST API query complete  200
REST API query started for 10.1101/2021.03.07.21253098...
REST API query complete  200
REST API query started for 10.1101/2021.02.26.21252096...
REST API query complete  200
REST API query started for 10.1101/2021.03.05.21252960...
REST API query complete  200
REST API query started for 10.1101/2021.05.11.21257011...
REST API query complete  200
REST API query started for 10.1101/2021.05.14.444076...
REST API query complete  200
REST API query started for 10.1101/2021.04.19.21255720...
REST API query complete  200
REST API query started for 10.1101/2021.01.27.21250617...
REST API query complete  200
REST API query started for 10.1101/2021.01.21.21250266...
REST API query complete  200
REST API query started for 10.1101/2021.02.27.4

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
0,10.1101/2021.05.18.21257267,1221,medRxiv,Infectious Diseases (except HIV/AIDS),True,Colchicine in patients admitted to hospital wi...,"[{'name': ' '}, {'name': 'Peter W Horby'}, {'n...",<sec><title>Background</title><p>Colchicine ha...,2021-05-18
1,10.1101/2021.05.20.21257461,220,medRxiv,Epidemiology,True,Efficacy Estimates for Various COVID-19 Vaccin...,"[{'name': 'Julia Shapiro'}, {'name': 'Natalie ...","<p>In this report, we provide summary estimate...",2021-05-21
2,10.1101/2021.05.17.444558,198,bioRxiv,Cell Biology,False,Polarized Dishevelled dissolution and condensa...,"[{'name': 'S. Zachary Swartz'}, {'name': 'Tzer...",<p>The organismal body axes that are formed du...,2021-05-18
3,10.1101/2021.05.17.444523,160,bioRxiv,Zoology,False,DiversityScanner: Robotic discovery of small i...,"[{'name': 'Lorenz Wührl'}, {'name': 'Christian...",<p>Invertebrate biodiversity remains poorly ex...,2021-05-18
4,10.1101/2021.05.20.445008,105,bioRxiv,Microbiology,True,Acute SARS-CoV-2 infection is associated with ...,"[{'name': 'Nicholas S. Rhoades'}, {'name': 'Am...",<p>Much of the research conducted on SARS-CoV-...,2021-05-20
5,10.1101/2021.05.19.444882,100,bioRxiv,Molecular Biology,True,The Structure of ChAdOx1/AZD-1222 Reveals Inte...,"[{'name': 'Alexander T. Baker'}, {'name': 'Rya...","<p>Adenovirus derived vectors, based on chimpa...",2021-05-20
6,10.1101/2021.05.17.444537,76,bioRxiv,Neuroscience,False,Attentional modulation of intrinsic timescales...,"[{'name': 'Roxana Zeraati'}, {'name': 'Yan-Lia...",<p>Neural activity fluctuates endogenously on ...,2021-05-18
7,10.1101/2021.05.11.21256972,55,medRxiv,Infectious Diseases (except HIV/AIDS),True,SARS-CoV-2 mRNA vaccines induce a greater arra...,"[{'name': 'Jéromine Klingler'}, {'name': 'Greg...",<sec><title>Background</title><p>Mass vaccinat...,2021-05-17
8,10.1101/2021.05.19.444823,54,bioRxiv,Microbiology,True,Bidirectional genome-wide CRISPR screens revea...,"[{'name': 'Antoine Rebendenne'}, {'name': 'Pri...",<p>Several genome-wide CRISPR knockout screens...,2021-05-19
9,10.1101/2021.05.21.445058,52,bioRxiv,Bioinformatics,False,Assessing and removing the effect of unwanted ...,"[{'name': 'Muhamad Fachrul'}, {'name': 'Guilla...",<p>Varying technologies and experimental appro...,2021-05-22


### Tweets of bioRxiv and medRxiv preprints

In [10]:
num_rows = tdf['archive'].count()
num_covid = tdf['covid'].value_counts(ascending=True)[1]
num_biorxiv = tdf['archive'].value_counts(ascending=True)[1]
num_medrxiv = tdf['archive'].value_counts(ascending=True)[0]
end_date = date.today().strftime('%Y-%m-%d')
max_count = tdf['tweets'].max()

md('{} preprints (including {} covering SARS-CoV-2, {} from bioRxiv and {} from medRxiv) published in the last 7 days before {} had been tweeted at least 5 times (maximum {}).'.format(num_rows, num_covid, num_biorxiv, num_medrxiv, end_date, max_count))

114 preprints (including 36 covering SARS-CoV-2, 82 from bioRxiv and 32 from medRxiv) published in the last 7 days before 2021-05-24 had been tweeted at least 5 times (maximum 1221).